In [ ]:
#@title
from IPython.display import Image
Image(url='https://i.imgur.com/DeB62Nl.png', width=500)

* [Kaggle Competition Page](https://www.kaggle.com/competitions/leeds-sciml-sea-ice-segmentation)
* [Weights & Biases SciML Leeds team](https://wandb.ai/sciml-leeds)

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/U4amljFGkiw" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

# Setup

First, let's set up the environment by installing the required libraries and importing the necessary modules.

* Create a Kaggle account: If you don't already have a Kaggle account, go to the Kaggle website (https://www.kaggle.com/) and sign up for an account.

* Generate the kaggle.json file: After logging in to Kaggle, go to your account settings page. Scroll down to the section titled "API" and click on the "Create New API Token" button. This will download a file named kaggle.json to your computer.

* Upload kaggle.json to the runtime environment. Use the file upload below.

In [ ]:
#@title
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    if fn != 'kaggle.json': print('Are you sure you\'ve uploaded the right file?')
    print(f'Uploaded file "{fn}"')

In [ ]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d spiruel/leeds-sciml-seaice

In [ ]:
!unzip leeds-sciml-seaice.zip

# 🚀 Installing and importing

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms.functional as TF
import glob
import os
from tqdm import tqdm
import numpy as np
import tifffile
from sklearn.model_selection import train_test_split

The `patch_paths` variables are arrays containing the names of the patch files.
Each patch patch has a corresponding SAR image, multispectral image, and label. (Except we've not given you the labels for the test patch paths).


In [16]:
DATA_DIR = 'sciml'
train_patch_paths = np.array([os.path.basename(i[:-9]) for i in glob.glob(f'{DATA_DIR}/train/*_sar.tiff')])
test_patch_paths = np.array([os.path.basename(i[:-9]) for i in glob.glob(f'{DATA_DIR}/test/*_sar.tiff')])

#display patch paths
train_patch_paths

array(['Patch20190104_218_1200_840', 'Patch20190205_75_1680_960',
       'Patch20191216_308_1560_1313', ..., 'Patch20190101_46_960_240',
       'Patch20190104_50_960_1200', 'Patch20191216_212_1080_1560'],
      dtype='<U27')

In [17]:
# Custom dataset class
class IceSegmentationDataset(Dataset):
    def __init__(self, patch_paths, data_dir, split='train'):
        self.patch_paths = patch_paths
        self.data_dir = data_dir
        self.split = split

    def __len__(self):
        return len(self.patch_paths)

    def __getitem__(self, index):
        patch_path = self.patch_paths[index]
        sar_path = os.path.join(self.data_dir, self.split, patch_path + '_sar.tiff')
        ms_path = os.path.join(self.data_dir, self.split, patch_path + '_vis.tiff')
        if self.split != 'test':
            label_path = os.path.join(self.data_dir, self.split, patch_path + '_ref.tiff')
            label_arr = self.load_img(label_path)
            y = label_arr

        sar_arr = self.load_img(sar_path)
        ms_arr = self.load_img(ms_path)

        X_sar = np.expand_dims(sar_arr, 0)[:, ::3, ::3]
        X_ms = ms_arr

        if self.split != 'test':
            return (X_sar, X_ms), y
        else:
            return (X_sar, X_ms), None

    def load_img(self, path):
        img = tifffile.imread(path)
        if len(img.shape) == 3:
          img = img.transpose((2, 0, 1))  # Transpose to match PyTorch tensor shape (C, H, W)
        return torch.from_numpy(img).float()

In [18]:
# Create data loaders
batch_size = 32
train_patch_paths, val_patch_paths = train_test_split(train_patch_paths, test_size=0.2, random_state=42)
train_dataset = IceSegmentationDataset(train_patch_paths, DATA_DIR)
val_dataset = IceSegmentationDataset(val_patch_paths, DATA_DIR)
test_dataset = IceSegmentationDataset(test_patch_paths, DATA_DIR, split='test')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [39]:
sar_int = np.array([])
ms_int_r = np.array([])
ms_int_g = np.array([])
ms_int_b = np.array([])

for i in range(len(train_dataset)):
    (X_sar, X_ms), y = train_dataset.__getitem__(i)
    sar_int = np.append(sar_int, X_sar.ravel())
    ms_int_r = np.append(ms_int_r, X_ms[0,:,:].ravel())
    ms_int_g = np.append(ms_int_g, X_ms[1,:,:].ravel())
    ms_int_b = np.append(ms_int_b, X_ms[2,:,:].ravel())

In [40]:
print(np.mean(ms_int_r), np.std(ms_int_r))
print(np.mean(ms_int_g), np.std(ms_int_g))
print(np.mean(ms_int_b), np.std(ms_int_b))
print(np.mean(sar_int), np.std(sar_int))

0.3556416964232872 0.5520505638405484
-0.2089460941649551 0.6538063835970725
-0.2977986399965673 0.6143507026534764
-0.801564234287415 0.16891951952490908


In [26]:
sar_hist = plt.hist(sar_int)

NameError: name 'plt' is not defined

In [ ]:
ms_hist = plt.hist(ms_int)

The U-Net model architecture is a convolutional neural network that consists of an encoder and a decoder.
The encoder extracts features from the input images, while the decoder upsamples the features to generate the final segmentation map.

The model is compiled with the Adam optimiser and binary cross-entropy loss.
During training, the validation data is used to monitor the performance of the model and early stopping is applied to prevent overfitting.

The training progress can be logged using the wandb library, which allows tracking and visualisation of metrics in a web interface.

In [ ]:
# Define the U-Net model architecture
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(4, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.bridge = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

        self.output = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid(),
            nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)
        )

    def forward(self, input_sar, input_ms):
        concat = torch.cat((input_sar, input_ms), dim=1)
        enc1 = self.encoder(concat)
        bridge = self.bridge(enc1)
        dec1 = self.decoder(bridge)
        output = self.output(dec1)
        return output

In [ ]:
# Initialise the model
model = UNet()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from segment_anything import sam_model_registry, SamPredictor
import torchvision.transforms as tf
from torch.nn.functional import threshold, normalize

In [ ]:
(sar, ms), label = train_dataset.__getitem__(0)
#sar = torch.tensor(sar)
#print(sar.shape, type(sar))
resize = tf.Resize((1024,1024))
ms = resize(ms)
print(ms.shape, type(ms))
#print(label.shape, type(label))

In [ ]:
# Define segment anything model

sam_checkpoint = "models/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

In [ ]:
# define optimiser & loss

optimiser = torch.optim.Adam(sam.mask_decoder.parameters())
criterion = nn.BCELoss()

In [ ]:
# define train loop

def train(sam_model, img, mask):
    
    with torch.no_grad():
        image_embedding = sam_model.image_encoder(img)
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
            points=None,
            boxes=None,
            masks=mask,
        )
    low_res_masks, iou_predictions = sam_model.mask_decoder(
      image_embeddings=image_embedding,
      image_pe=sam_model.prompt_encoder.get_dense_pe(),
      sparse_prompt_embeddings=sparse_embeddings,
      dense_prompt_embeddings=dense_embeddings,
      multimask_output=False,
    )
    upscaled_masks = sam_model.postprocess_masks(low_res_masks, input_size, original_image_size).to(device)
    binary_mask = normalize(threshold(upscaled_masks, 0.0, 0)).to(device)
    loss = loss_fn(binary_mask, gt_binary_mask)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# define train loop

def evaluate(sam_model, img, gt_mask):
    
    with torch.no_grad():
        image_embedding = sam_model.image_encoder(img)
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
    low_res_masks, iou_predictions = sam_model.mask_decoder(
      image_embeddings=image_embedding,
      image_pe=sam_model.prompt_encoder.get_dense_pe(),
      sparse_prompt_embeddings=sparse_embeddings,
      dense_prompt_embeddings=dense_embeddings,
      multimask_output=False,
    )
    upscaled_masks = sam_model.postprocess_masks(low_res_masks, input_size, original_image_size).to(device)
    prediction = normalize(threshold(upscaled_masks, 0.0, 0)).to(device)
    
    return prediction

# ✅ Sign Up

Sign up to a free [Weights & Biases account here](https://wandb.ai/signup)

[Weights and Biases docs](https://docs.wandb.ai/quickstart)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (input_sar, input_ms), labels in tqdm(train_loader):
        input_sar = input_sar.to(device)
        input_ms = input_ms.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(input_sar, input_ms)
        outputs = outputs[:,:,::4,::4].squeeze()
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * input_sar.size(0)

    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for (input_sar, input_ms), labels in tqdm(val_loader):
            input_sar = input_sar.to(device)
            input_ms = input_ms.to(device)
            labels = labels.to(device)

            outputs = model(input_sar, input_ms)
            outputs = outputs[:,:,::4,::4].squeeze()

            loss = criterion(outputs, labels)

            val_loss += loss.item() * input_sar.size(0)

    val_loss /= len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")




# # Initialize a new W&B run
# wandb_enabled = False

# earlystopper = EarlyStopping(patience=5, verbose=1)
# if wandb_enabled:
#     wandb.init(entity='sciml-leeds', project='sea-ice-segmentation')
#     history = model.fit(train_generator,
#           validation_data=val_generator,
#           epochs=10,
#           use_multiprocessing=True,
#           workers=6,
#           callbacks=[earlystopper, WandbCallback()])
#     wandb.finish()
# else:
#     history = model.fit(train_generator,
#           validation_data=val_generator,
#           epochs=10,
#           use_multiprocessing=True,
#           workers=6,
#           callbacks=[earlystopper])

In [ ]:
# Test
model.eval()
predictions = []
with torch.no_grad():
    for (input_sar, input_ms), _ in test_loader:
        input_sar = input_sar.to(device)
        input_ms = input_ms.to(device)

        outputs = model(input_sar, input_ms)
        predictions.append(outputs.cpu().numpy())

predictions = np.concatenate(predictions)

# Convert predictions to binary mask
predictions = (predictions > 0.5).astype(np.uint8)


In [ ]:
#@title Now plot the results of the training run:

metric_to_plot = 'loss' #@param ['loss', 'accuracy', 'mean_io_u']

ys = history.history[metric_to_plot]
ys_val = history.history['val_'+metric_to_plot]
num_epochs = len(ys)
plt.plot(range(num_epochs), ys, label='train')
plt.plot(range(num_epochs), ys_val, label='val')
plt.legend()
plt.ylabel(metric_to_plot)

Let's dive into the visualization of our sea ice segmentation results using the validation dataset. By running the code snippet provided, we obtain valuable insights into the model's performance and its ability to accurately identify sea ice boundaries.

First, the model makes predictions on the validation dataset using the model.predict() function. The resulting predictions are stored in the variable preds. These predictions represent the segmentation masks, indicating the presence or absence of sea ice in the images.

Next, we extract a single batch of data from the validation generator using next(iter(val_generator)). This batch consists of SAR (Synthetic Aperture Radar) images, multispectral (MS) images, and their corresponding ground truth segmentation masks. These components are unpacked into the variables X_sar_test, X_ms_test, and y_test, respectively.

Now, we proceed to visualize the results. Within a loop, we iterate over the predictions stored in preds. For each prediction, we create a figure with a grid layout of four subplots.

The first subplot, labeled "SAR Image," displays the SAR image from the validation set. SAR imagery provides valuable information about the surface characteristics of sea ice.

The second subplot, labeled "MS Image," showcases the multispectral image captured during the validation process. Multispectral data enables us to analyze sea ice from different spectral perspectives.

Moving on to the third subplot, titled "Segmentation Mask," we observe the model's predicted segmentation mask. The thresholding operation (`preds[i,:,:,0]>threshold`) helps us visualize the predicted sea ice boundaries clearly.

Lastly, the fourth subplot, labeled "Ground Truth," presents the actual ground truth segmentation mask obtained from the validation dataset. This serves as a reference to assess the accuracy of our model's predictions.

By carefully examining these subplots, we can compare the model's predictions against the ground truth and gain insights into the performance and limitations of our sea ice segmentation model.

In [ ]:
#@markdown You need to specify the threshold value, which determines the threshold for classifying pixels as foreground or background. Adjusting this value may affect the performance of the segmentation.

threshold = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}

In [ ]:
preds = model.predict(val_generator,steps=1)

(X_sar_test, X_ms_test), y_test = next(iter(val_generator))

for i in range(preds.shape[0]):
    plt.figure(figsize=(10, 5))
    plt.subplot(141)
    plt.imshow(X_sar_test[i,:,:,:])
    plt.title('SAR Image')
    plt.axis('off')
    plt.subplot(142)
    plt.imshow(X_ms_test[i,:,:,:])
    plt.title('MS Image')
    plt.axis('off')
    plt.subplot(143)
    plt.imshow(preds[i,:,:,0]>threshold)
    plt.title('Segmentation Mask')
    plt.axis('off')
    plt.subplot(144)
    plt.imshow(y_test[i,:,:,0])
    plt.title('Ground Truth')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


* Are there any notable differences between the predicted segmentation mask and the ground truth?
* How does adjusting the threshold value impact the visualisation of the segmentation mask?
* Can you identify any challenging areas where the model struggles to accurately predict sea ice boundaries?
* What potential implications can accurate sea ice segmentation have on environmental research and decision-making processes?

# Generate Submission
Ta-da! 📝🎉 Our model is trained and ready, and it's time to run our predictions on the test set and create the submission file.

A submission file named 'submission.csv' is created to store the predictions.
For each image in the test set, the image ID and run-length encoded pixels are written to the file.

In [ ]:
import pandas as pd

#ref:https://www.kaggle.com/paulorzp/run-length-encode-and-decode.
def rle_encode(mask):
    '''
    mask: numpy array binary mask
    1 - mask
    0 - background
    Returns encoded run length
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Predict on test data
predictions = model.predict(test_generator)

# Generate submission DataFrame
submission_df = pd.DataFrame(columns=['ImageId', 'EncodedPixels'])

for i, patch_path in enumerate(test_patch_paths):
    image_id = os.path.basename(patch_path)
    mask = predictions[i, :, :, 0] > threshold
    encoded_pixels = rle_encode(mask)
    submission_df.loc[i] = [image_id, encoded_pixels]

# Save submission DataFrame to CSV file
submission_df.to_csv('submission.csv', index=False)

In [ ]:
#@title Click to download submission file

from google.colab import files
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Download csv", width=200, height=200)
output = widgets.Output()

def on_button_clicked(b):
    files.download('submission.csv')

button.on_click(on_button_clicked)
display(button, output)
